<a href="https://colab.research.google.com/github/vajihe-ameri/predict-software-bugs-in-java-classes/blob/main/tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install requirements

In [1]:
!pip install sklearn pandas
!pip install prettytable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=3265d86e8d331a2c28baabae4f499e3bb95de720da9f607dd202149957b390e8
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Read dataset

In [13]:
import pandas as pd
from google.colab import drive 
drive = drive.mount('/content/drive') 

Mounted at /content/drive


Call algorithms from sklearntrain_featurestrain_fea

In [20]:
from prettytable import PrettyTable
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.tree import export_graphviz
from IPython import display
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

projects = ['oryx']

for project in projects:
  print(f'Project => {project}')
  '''Read data from CSV with Pandas'''
  _file =f"drive/MyDrive/subtract/{project}/class.csv"

  df = pd.read_csv(_file)
  print('information of file:', df.describe(include="all"))
  print('-------------------------------------------------------------------------------')
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))
  df = pd.read_csv(_file, usecols = cols)
  print('delete data NAN:',df.describe(include="all"))
  print('-------------------------------------------------------------------------------')
  df.pop('LongName')
  df.pop('Hash')
  print('delete cols of string:',df.describe(include="all"))
  print('-------------------------------------------------------------------------------')
  df = df.T.drop_duplicates().T
  print('delete cols by same data:',df.describe(include="all"))
  print('-------------------------------------------------------------------------------')
  df = (df-df.mean())/df.std()
  print('data normalize:',df.describe(include="all"))
  print('-------------------------------------------------------------------------------')

Project => oryx
information of file:                                             Hash  \
count                                        598   
unique                                        87   
top     d16f89b25fc5e12f94af6432274c66e0394c1743   
freq                                          45   
mean                                         NaN   
std                                          NaN   
min                                          NaN   
25%                                          NaN   
50%                                          NaN   
75%                                          NaN   
max                                          NaN   

                                           LongName          CC         CCL  \
count                                           598  598.000000  598.000000   
unique                                          194         NaN         NaN   
top     com.cloudera.oryx.lambda.AbstractSparkLayer         NaN         NaN   
freq                  

random forest

In [23]:
print('Random forest')
projects = ['oryx']

for project in projects:
  print(f'Project: {project}')

  '''Read data from CSV with Pandas'''
  _file =f"drive/MyDrive/subtract/{project}/class.csv"
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))

  df = pd.read_csv(_file, usecols =cols)
  df.pop('LongName')
  df.pop('Hash')
  df = df.T.drop_duplicates().T

  '''Split into train and test with numpy (0.75 train, 0.25 test)'''
  from sklearn.model_selection import train_test_split
  train, test = train_test_split(df, test_size=0.25)

  df=(df-df.mean())/df.std()

  test_target = test.pop('Number of Bugs')
  test_features = test
 
  train_target = train.pop('Number of Bugs')
  train_features = train
  
  '''Create random forest classifier'''
  model1 = RandomForestClassifier()
  model2 = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_depth = 1, random_state = 80)
  model3 = RandomForestClassifier(n_estimators=90, criterion = 'entropy' , max_depth = 10, random_state = 50)
  model4 = RandomForestClassifier(n_estimators=50, criterion = 'entropy', max_depth = 20, random_state = 60)
  model5 = RandomForestClassifier(n_estimators=1, criterion = 'gini', max_depth = 30, random_state = 40)

  '''Train and test'''
  model1.fit(train_features, train_target)
  model2.fit(train_features, train_target)
  model3.fit(train_features, train_target)
  model4.fit(train_features, train_target)
  model5.fit(train_features, train_target)

  preds1 = model1.predict(test_features)
  preds2 = model2.predict(test_features)
  preds3 = model3.predict(test_features)
  preds4 = model4.predict(test_features)
  preds5 = model5.predict(test_features)

  cr1 = confusion_matrix(test_target, preds1)
  cr2 = confusion_matrix(test_target, preds2)
  cr3 = confusion_matrix(test_target, preds3)
  cr4 = confusion_matrix(test_target, preds4)
  cr5 = confusion_matrix(test_target, preds5)

  from sklearn.metrics import f1_score

  f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
  f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
  f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
  f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')
  f1_weighted5 = f1_score(test_target, preds5, average = 'weighted')

  f1_macro1 = f1_score(test_target, preds1, average = 'macro')
  f1_macro2 = f1_score(test_target, preds2, average = 'macro')
  f1_macro3 = f1_score(test_target, preds3, average = 'macro')
  f1_macro4 = f1_score(test_target, preds4, average = 'macro')
  f1_macro5 = f1_score(test_target, preds5, average = 'macro')

  f1_micro1 = f1_score(test_target, preds1, average = 'micro')
  f1_micro2 = f1_score(test_target, preds2, average = 'micro')
  f1_micro3 = f1_score(test_target, preds3, average = 'micro')
  f1_micro4 = f1_score(test_target, preds4, average = 'micro')
  f1_micro5 = f1_score(test_target, preds5, average = 'micro')

  f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
  f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
  f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
  f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
  f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)

  f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
  f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
  f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
  f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
  f1_score_macro5 = "%.2f"%round(f1_macro5, 2)

  f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
  f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
  f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
  f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
  f1_score_micro5 = "%.2f"%round(f1_micro5, 2)

  head = PrettyTable(['random_state', 'n_estimatorsint', 'criterion', 'max_depth', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
  head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
  head.add_row(['80', '10', 'gini', '1', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
  head.add_row(['50', '100', 'entropy', '10', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
  head.add_row(['60', '50', 'entropy', '20', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
  head.add_row(['40', '1', 'gini', '30', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5]) 
  print(head)

Random forest
Project: oryx
+--------------+-----------------+-----------+-----------+------------------+-------------------+----------------+----------------+
| random_state | n_estimatorsint | criterion | max_depth | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+-----------------+-----------+-----------+------------------+-------------------+----------------+----------------+
|              |                 |           |           |  [[128   5   2]  |        0.85       |      0.37      |      0.87      |
|              |                 |           |           |   [ 13   2   0]  |                   |                |                |
|              |                 |           |           |  [  0   0   0]]  |                   |                |                |
|      80      |        10       |    gini   |     1     |    [[135   0]    |        0.85       |      0.47      |      0.90      |
|              |                 |           |  

logistic regression

In [24]:
print('Logistic regression')
projects = ['oryx']

for project in projects:
  print(f'Project: {project}')

  '''Read data from CSV with Pandas'''
  _file =f"drive/MyDrive/subtract/{project}/class.csv"
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))

  df = pd.read_csv(_file, usecols =cols)
  df.pop('LongName')
  df.pop('Hash')
  df = df.T.drop_duplicates().T

  '''Split into train and test with numpy (0.75 train, 0.25 test)'''
  from sklearn.model_selection import train_test_split
  train, test = train_test_split(df, test_size=0.25)

  df=(df-df.mean())/df.std()

  test_target = test.pop('Number of Bugs')
  test_features = test
 
  train_target = train.pop('Number of Bugs')
  train_features = train
  
  '''Create logistic regression classifier'''  
  model1 = LogisticRegression()
  model2 = LogisticRegression(solver='sag', penalty='l2',n_jobs=-1, random_state=10)
  model3 = LogisticRegression(solver='saga', penalty='l2',n_jobs=-1, random_state=20)
  model4 = LogisticRegression(solver='liblinear', penalty='l1',n_jobs=-1, random_state=0)
  model5 = LogisticRegression(solver='liblinear', penalty='l2',n_jobs=-1, random_state=90)
  model6 = LogisticRegression(solver='lbfgs', penalty='l2',n_jobs=-1, random_state=50)
  model7 = LogisticRegression(solver='newton-cg', penalty='l2',n_jobs=-1, random_state=90)

  '''Train and test'''
  model1.fit(train_features, train_target)
  model2.fit(train_features, train_target)
  model3.fit(train_features, train_target)
  model4.fit(train_features, train_target)
  model5.fit(train_features, train_target)
  model6.fit(train_features, train_target)
  model7.fit(train_features, train_target)

  preds1 = model1.predict(test_features)
  preds2 = model2.predict(test_features)
  preds3 = model3.predict(test_features)
  preds4 = model4.predict(test_features)
  preds5 = model5.predict(test_features)
  preds6 = model5.predict(test_features)
  preds7 = model5.predict(test_features)

  cr1 = confusion_matrix(test_target, preds1)
  cr2 = confusion_matrix(test_target, preds2)
  cr3 = confusion_matrix(test_target, preds3)
  cr4 = confusion_matrix(test_target, preds4)
  cr5 = confusion_matrix(test_target, preds5)
  cr6 = confusion_matrix(test_target, preds5)
  cr7 = confusion_matrix(test_target, preds5)

  from sklearn.metrics import f1_score

  f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
  f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
  f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
  f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')
  f1_weighted5 = f1_score(test_target, preds5, average = 'weighted')
  f1_weighted6 = f1_score(test_target, preds5, average = 'weighted')
  f1_weighted7 = f1_score(test_target, preds5, average = 'weighted')

  f1_macro1 = f1_score(test_target, preds1, average = 'macro')
  f1_macro2 = f1_score(test_target, preds2, average = 'macro')
  f1_macro3 = f1_score(test_target, preds3, average = 'macro')
  f1_macro4 = f1_score(test_target, preds4, average = 'macro')
  f1_macro5 = f1_score(test_target, preds5, average = 'macro')
  f1_macro6 = f1_score(test_target, preds5, average = 'macro')
  f1_macro7 = f1_score(test_target, preds5, average = 'macro')

  f1_micro1 = f1_score(test_target, preds1, average = 'micro')
  f1_micro2 = f1_score(test_target, preds2, average = 'micro')
  f1_micro3 = f1_score(test_target, preds3, average = 'micro')
  f1_micro4 = f1_score(test_target, preds4, average = 'micro')
  f1_micro5 = f1_score(test_target, preds5, average = 'micro')
  f1_micro6 = f1_score(test_target, preds5, average = 'micro')
  f1_micro7 = f1_score(test_target, preds5, average = 'micro')
 
  f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
  f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
  f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
  f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
  f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)
  f1_score_weighted6 = "%.2f"%round(f1_weighted6, 2)
  f1_score_weighted7 = "%.2f"%round(f1_weighted7, 2)

  f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
  f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
  f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
  f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
  f1_score_macro5 = "%.2f"%round(f1_macro5, 2)
  f1_score_macro6 = "%.2f"%round(f1_macro6, 2)
  f1_score_macro7 = "%.2f"%round(f1_macro7, 2)

  f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
  f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
  f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
  f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
  f1_score_micro5 = "%.2f"%round(f1_micro5, 2)
  f1_score_micro6 = "%.2f"%round(f1_micro6, 2)
  f1_score_micro7 = "%.2f"%round(f1_micro7, 2)

  head = PrettyTable(['random_state', 'solver', 'penalty', 'n_jobs', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
  head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
  head.add_row(['10', 'sag', 'l2', '-1', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
  head.add_row(['20', 'saga', 'l2', '-1', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
  head.add_row(['0', 'liblinear', 'l1', '-1', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
  head.add_row(['90', 'liblinear', 'l2', '-1', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5]) 
  head.add_row(['50', 'lbfgs', 'l2', '-1', cr6, f1_score_weighted6, f1_score_macro6, f1_score_micro6]) 
  head.add_row(['90', 'newton-cg', 'l2', '-1', cr7, f1_score_weighted7, f1_score_macro7, f1_score_micro7]) 
  print(head)

Logistic regression
Project: oryx


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1526: UserWarning: 'n_jobs' > 1 does not h

+--------------+-----------+---------+--------+------------------+-------------------+----------------+----------------+
| random_state |   solver  | penalty | n_jobs | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+-----------+---------+--------+------------------+-------------------+----------------+----------------+
|              |           |         |        |  [[128   4   1]  |        0.84       |      0.37      |      0.87      |
|              |           |         |        |   [ 15   2   0]  |                   |                |                |
|              |           |         |        |  [  0   0   0]]  |                   |                |                |
|      10      |    sag    |    l2   |   -1   |    [[132   1]    |        0.83       |      0.47      |      0.88      |
|              |           |         |        |    [ 17   0]]    |                   |                |                |
|      20      |    saga   |    

naive bayes

In [25]:
print('Naive bayes')
projects = ['oryx']

for project in projects:
  print(f'Project: {project}')

  '''Read data from CSV with Pandas'''

  _file =f"drive/MyDrive/subtract/{project}/class.csv"
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))

  df = pd.read_csv(_file, usecols =cols)
  df.pop('LongName')
  df.pop('Hash')
  df = df.T.drop_duplicates().T

  '''Split into train and test with numpy (0.75 train, 0.25 test)'''
  from sklearn.model_selection import train_test_split
  train, test = train_test_split(df, test_size=0.25)

  df=(df-df.mean())/df.std()

  test_target = test.pop('Number of Bugs')
  test_features = test
 
  train_target = train.pop('Number of Bugs')
  train_features = train
  
  '''Create random forest classifier'''
  model1 = GaussianNB()
  model2 = GaussianNB(var_smoothing = 1e-11)
  model3 = GaussianNB(var_smoothing = 1e-09)
  model4 = GaussianNB(var_smoothing = 1e-10)

  '''Train and test'''
  model1.fit(train_features, train_target)
  model2.fit(train_features, train_target)
  model3.fit(train_features, train_target)
  model4.fit(train_features, train_target)

  preds1 = model1.predict(test_features)
  preds2 = model2.predict(test_features)
  preds3 = model3.predict(test_features)
  preds4 = model4.predict(test_features)

  cr1 = confusion_matrix(test_target, preds1)
  cr2 = confusion_matrix(test_target, preds2)
  cr3 = confusion_matrix(test_target, preds3)
  cr4 = confusion_matrix(test_target, preds4)

  from sklearn.metrics import f1_score

  f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
  f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
  f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
  f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')

  f1_macro1 = f1_score(test_target, preds1, average = 'macro')
  f1_macro2 = f1_score(test_target, preds2, average = 'macro')
  f1_macro3 = f1_score(test_target, preds3, average = 'macro')
  f1_macro4 = f1_score(test_target, preds4, average = 'macro')

  f1_micro1 = f1_score(test_target, preds1, average = 'micro')
  f1_micro2 = f1_score(test_target, preds2, average = 'micro')
  f1_micro3 = f1_score(test_target, preds3, average = 'micro')
  f1_micro4 = f1_score(test_target, preds4, average = 'micro')

  f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
  f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
  f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
  f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)

  f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
  f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
  f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
  f1_score_macro4 = "%.2f"%round(f1_macro4, 2)

  f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
  f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
  f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
  f1_score_micro4 = "%.2f"%round(f1_micro4, 2)

  head = PrettyTable(['var_smoothing', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
  head.add_row(['', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
  head.add_row(['1e-11', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
  head.add_row(['1e-9', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
  head.add_row(['1e-10', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
  print(head)

Naive bayes
Project: oryx
+---------------+------------------+-------------------+----------------+----------------+
| var_smoothing | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+---------------+------------------+-------------------+----------------+----------------+
|               |     [[98 33]     |        0.78       |      0.62      |      0.74      |
|               |     [ 6 13]]     |                   |                |                |
|     1e-11     |     [[71 60]     |        0.65       |      0.53      |      0.59      |
|               |     [ 2 17]]     |                   |                |                |
|      1e-9     |     [[98 33]     |        0.78       |      0.62      |      0.74      |
|               |     [ 6 13]]     |                   |                |                |
|     1e-10     |     [[85 46]     |        0.72       |      0.56      |      0.66      |
|               |     [ 5 14]]     |                   |        

Multi-Layer Perceptron Neural Network

In [26]:
print('Multi layer perceptron neural network')
projects = ['oryx']

for project in projects:
  print(f'Project: {project}')

  '''Read data from CSV with Pandas'''
  _file =f"drive/MyDrive/subtract/{project}/class.csv"
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))

  df = pd.read_csv(_file, usecols =cols)
  df.pop('LongName')
  df.pop('Hash')
  df = df.T.drop_duplicates().T

  '''Split into train and test with numpy (0.75 train, 0.25 test)'''
  from sklearn.model_selection import train_test_split
  train, test = train_test_split(df, test_size=0.25)

  df=(df-df.mean())/df.std()

  test_target = test.pop('Number of Bugs')
  test_features = test
 
  train_target = train.pop('Number of Bugs')
  train_features = train
  
  '''Create Artificial Neural Network classifier'''
  model1 = MLPClassifier()
  model2 = MLPClassifier(solver='lbfgs', alpha=0.01, hidden_layer_sizes=(5, 2), activation = 'tanh', random_state=10)
  model3 = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(150, 100, 50), activation = 'logistic',random_state=1)
  model4 = MLPClassifier(solver='adam', alpha=0.05, hidden_layer_sizes=(100, 50, 30), activation = 'relu', random_state=90)
  model5 = MLPClassifier(solver='lbfgs', alpha=0.0001, hidden_layer_sizes=(120, 80, 40), activation = 'identity', random_state=40)

  '''Train and test'''
  model1.fit(train_features, train_target)
  model2.fit(train_features, train_target)
  model3.fit(train_features, train_target)
  model4.fit(train_features, train_target)
  model5.fit(train_features, train_target)

  preds1 = model1.predict(test_features)
  preds2 = model2.predict(test_features)
  preds3 = model3.predict(test_features)
  preds4 = model4.predict(test_features)
  preds5 = model5.predict(test_features)

  cr1 = confusion_matrix(test_target, preds1)
  cr2 = confusion_matrix(test_target, preds2)
  cr3 = confusion_matrix(test_target, preds3)
  cr4 = confusion_matrix(test_target, preds4)
  cr5 = confusion_matrix(test_target, preds5)

  from sklearn.metrics import f1_score

  f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
  f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
  f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
  f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')
  f1_weighted5 = f1_score(test_target, preds5, average = 'weighted')

  f1_macro1 = f1_score(test_target, preds1, average = 'macro')
  f1_macro2 = f1_score(test_target, preds2, average = 'macro')
  f1_macro3 = f1_score(test_target, preds3, average = 'macro')
  f1_macro4 = f1_score(test_target, preds4, average = 'macro')
  f1_macro5 = f1_score(test_target, preds5, average = 'macro')

  f1_micro1 = f1_score(test_target, preds1, average = 'micro')
  f1_micro2 = f1_score(test_target, preds2, average = 'micro')
  f1_micro3 = f1_score(test_target, preds3, average = 'micro')
  f1_micro4 = f1_score(test_target, preds4, average = 'micro')
  f1_micro5 = f1_score(test_target, preds5, average = 'micro')

  f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
  f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
  f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
  f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
  f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)

  f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
  f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
  f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
  f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
  f1_score_macro5 = "%.2f"%round(f1_macro5, 2)

  f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
  f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
  f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
  f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
  f1_score_micro5 = "%.2f"%round(f1_micro5, 2)

  head = PrettyTable(['random_state', 'activation', 'solver', 'hidden_layer_sizestuple', 'alpha', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
  head.add_row(['', '', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
  head.add_row(['10', 'tanh', 'lbfgs', '(5, 2)', '0.01', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
  head.add_row(['1', 'logistic', 'sgd', '(150, 100, 50)', 'le-5', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
  head.add_row(['90', 'relu', 'adam', '(100, 50, 30)', '0.05', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
  head.add_row(['40', 'identity', 'lbfgs', '(120, 80, 40)', '0.0001', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5]) 
  print(head)

Multi layer perceptron neural network
Project: oryx
+--------------+------------+--------+-------------------------+--------+------------------+-------------------+----------------+----------------+
| random_state | activation | solver | hidden_layer_sizestuple | alpha  | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+------------+--------+-------------------------+--------+------------------+-------------------+----------------+----------------+
|              |            |        |                         |        |  [[127   8   1]  |        0.85       |      0.36      |      0.86      |
|              |            |        |                         |        |   [ 12   2   0]  |                   |                |                |
|              |            |        |                         |        |  [  0   0   0]]  |                   |                |                |
|      10      |    tanh    | lbfgs  |          (5, 2)         |  

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Support vector machine

In [27]:
print('Support vector machine')
projects = ['oryx']

for project in projects:
  print(f'Project: {project}')

  '''Read data from CSV with Pandas'''
  _file =f"drive/MyDrive/subtract/{project}/class.csv"
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))

  df = pd.read_csv(_file, usecols =cols)
  df.pop('LongName')
  df.pop('Hash')
  df = df.T.drop_duplicates().T

  '''Split into train and test with numpy (0.75 train, 0.25 test)'''
  from sklearn.model_selection import train_test_split
  train, test = train_test_split(df, test_size=0.25)

  df=(df-df.mean())/df.std()

  test_target = test.pop('Number of Bugs')
  test_features = test
 
  train_target = train.pop('Number of Bugs')
  train_features = train
  
  '''Create Artificial Neural Network classifier'''
  model1 = SVC()
  model2 = SVC(kernel='sigmoid', C=100, gamma=1, random_state=85)
  model3 = SVC(kernel='rbf', C=0.1, gamma=0.01, random_state=15)
  model4 = SVC(kernel='linear', C=1, gamma=0.001, random_state=74)

  '''Train and test'''
  model1.fit(train_features, train_target)
  model2.fit(train_features, train_target)
  model3.fit(train_features, train_target)
  model4.fit(train_features, train_target)

  preds1 = model1.predict(test_features)
  preds2 = model2.predict(test_features)
  preds3 = model3.predict(test_features)
  preds4 = model4.predict(test_features)

  cr1 = confusion_matrix(test_target, preds1)
  cr2 = confusion_matrix(test_target, preds2)
  cr3 = confusion_matrix(test_target, preds3)
  cr4 = confusion_matrix(test_target, preds4)

  from sklearn.metrics import f1_score

  f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
  f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
  f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
  f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')

  f1_macro1 = f1_score(test_target, preds1, average = 'macro')
  f1_macro2 = f1_score(test_target, preds2, average = 'macro')
  f1_macro3 = f1_score(test_target, preds3, average = 'macro')
  f1_macro4 = f1_score(test_target, preds4, average = 'macro')

  f1_micro1 = f1_score(test_target, preds1, average = 'micro')
  f1_micro2 = f1_score(test_target, preds2, average = 'micro')
  f1_micro3 = f1_score(test_target, preds3, average = 'micro')
  f1_micro4 = f1_score(test_target, preds4, average = 'micro')

  f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
  f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
  f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
  f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)

  f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
  f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
  f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
  f1_score_macro4 = "%.2f"%round(f1_macro4, 2)

  f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
  f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
  f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
  f1_score_micro4 = "%.2f"%round(f1_micro4, 2)

  head = PrettyTable(['random_state', 'c', 'gamma', 'kernel', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
  head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
  head.add_row(['85', '100', '1', 'sigmoid', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
  head.add_row(['15', '0.01', '0.01', 'rbf', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
  head.add_row(['74', '1', '0.001', 'linear', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
  print(head)

Support vector machine
Project: oryx
+--------------+------+-------+---------+------------------+-------------------+----------------+----------------+
| random_state |  c   | gamma |  kernel | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+------+-------+---------+------------------+-------------------+----------------+----------------+
|              |      |       |         |    [[134   0]    |        0.84       |      0.47      |      0.89      |
|              |      |       |         |    [ 16   0]]    |                   |                |                |
|      85      | 100  |   1   | sigmoid |    [[134   0]    |        0.84       |      0.47      |      0.89      |
|              |      |       |         |    [ 16   0]]    |                   |                |                |
|      15      | 0.01 |  0.01 |   rbf   |    [[134   0]    |        0.84       |      0.47      |      0.89      |
|              |      |       |         |  

decision tree

In [28]:
print('Decision tree')
projects = ['oryx']

for project in projects:
  print(f'Project: {project}')

  '''Read data from CSV with Pandas'''
  _file =f"drive/MyDrive/subtract/{project}/class.csv"
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))

  df = pd.read_csv(_file, usecols =cols)
  df.pop('LongName')
  df.pop('Hash')
  df = df.T.drop_duplicates().T

  '''Split into train and test with numpy (0.75 train, 0.25 test)'''
  from sklearn.model_selection import train_test_split
  train, test = train_test_split(df, test_size=0.25)

  df=(df-df.mean())/df.std()

  test_target = test.pop('Number of Bugs')
  test_features = test
 
  train_target = train.pop('Number of Bugs')
  train_features = train

  head = PrettyTable(['random_state', 'max_depth', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
  
  max_depth_range = list(range(1, 15))
  for depth in max_depth_range:

    '''Create decision tree classifier'''    
    model = tree.DecisionTreeClassifier(max_depth = depth,  random_state = 90)
  
    '''Train and test'''
    model.fit(train_features, train_target)
    preds = model.predict(test_features)

    cr = confusion_matrix(test_target, preds)

    from sklearn.metrics import f1_score

    f1_weighted = f1_score(test_target, preds, average = 'weighted')
    f1_macro = f1_score(test_target, preds, average = 'macro')
    f1_micro = f1_score(test_target, preds, average = 'micro')

    f1_score_weighted = "%.2f"%round(f1_weighted, 2)
    f1_score_macro = "%.2f"%round(f1_macro, 2)
    f1_score_micro = "%.2f"%round(f1_micro, 2)

    head.add_row(['90', depth, cr, f1_score_weighted, f1_score_macro, f1_score_micro])
  print(head)
  


Decision tree
Project: oryx
+--------------+-----------+------------------+-------------------+----------------+----------------+
| random_state | max_depth | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+-----------+------------------+-------------------+----------------+----------------+
|      90      |     1     |    [[134   0]    |        0.84       |      0.47      |      0.89      |
|              |           |    [ 16   0]]    |                   |                |                |
|      90      |     2     |    [[134   0]    |        0.84       |      0.47      |      0.89      |
|              |           |    [ 16   0]]    |                   |                |                |
|      90      |     3     |    [[133   1]    |        0.87       |      0.58      |      0.90      |
|              |           |    [ 14   2]]    |                   |                |                |
|      90      |     4     |    [[129   5]    |       